# CLIMBRA CLIMA

> Notebook organizado para reprodutibilidade. Edite apenas a célula **CONFIGURAÇÕES**.

In [ ]:
from pathlib import Path
import os

# CONFIGURAÇÕES (edite se necessário)
# A pasta raiz do projeto (por padrão, a pasta acima de /notebooks)
ROOT = Path(os.getenv('CLIMBRA_PROJECT_ROOT', Path.cwd().parent)).resolve()
DATA_DIR = ROOT / 'data'
RAW_DIR  = DATA_DIR / '00_raw'
INT_DIR  = DATA_DIR / '01_intermediate'
FINAL_DIR= DATA_DIR / '02_final'
OUT_DIR  = ROOT / 'outputs'
FIG_DIR  = OUT_DIR / 'figures'
TAB_DIR  = OUT_DIR / 'tables'

for d in [RAW_DIR, INT_DIR, FINAL_DIR, FIG_DIR, TAB_DIR]:
    d.mkdir(parents=True, exist_ok=True)


In [ ]:
# ------------------------------------------------------------------------------
# Script: Extrair e Recortar Dados Climáticos de Arquivos NetCDF (.nc)
# ------------------------------------------------------------------------------
# Descrição:
# Este script realiza a leitura de arquivos NetCDF contendo dados climáticos
# (por exemplo, precipitação, temperatura, etc.), aplica um recorte espacial 
# com base em um shapefile (ex: bacia hidrográfica) e exporta os dados 
# espacialmente recortados para arquivos CSV individuais.
#
# Etapas:
# 1. Define caminhos de entrada e saída, e variável de interesse (ex: 'pr').
# 2. Garante que o shapefile esteja no mesmo sistema de coordenadas (WGS84).
# 3. Lê cada arquivo NetCDF da pasta e aplica o recorte espacial.
# 4. Converte os dados recortados para DataFrame e remove valores nulos.
# 5. Salva um arquivo CSV por ponto do grid com nome <modelo>_<lat>_<lon>.csv
#
# Requisitos:
# - Pacotes: xarray, rioxarray, geopandas, pandas, tqdm
# - Sistema de coordenadas: EPSG:4326 (WGS84) no shapefile e nos NetCDFs
#
# Autor: Matheus Marinho
# Projeto: Mestrado - Modelagem Hidrológica e Projeções CLIMBra
# Data: Junho/2025
# ------------------------------------------------------------------------------
import xarray as xr
import geopandas as gpd
import rioxarray
import pandas as pd
import os
from tqdm import tqdm

# === CONFIGURAÇÕES ===
pasta_nc = r"E:\Banco de Dados_CLIMBRA\SSP5_85\Tmin"
caminho_saida = r"E:\CLIMBRA_SSP585\Tmin"
shapefile = r"E:\IGUAÇU_OTTO\Shp\Buffer.shp"
variavel = "tasmin"

# === Ler shapefile ===
bacia = gpd.read_file(shapefile)
bacia = bacia.to_crs("EPSG:4326")

# === Lista de arquivos ===
arquivos_nc = [os.path.join(pasta_nc, f) for f in os.listdir(pasta_nc) if f.endswith(".nc")]

# === Processamento ===
for caminho_nc in tqdm(arquivos_nc, desc="Processando arquivos"):

    try:
        nome_modelo = os.path.basename(caminho_nc).replace(".nc", "")
        pasta_modelo = os.path.join(caminho_saida, nome_modelo)
        os.makedirs(pasta_modelo, exist_ok=True)

        ds = xr.open_dataset(caminho_nc)
        dados = ds[variavel]
        dados.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
        dados.rio.write_crs("EPSG:4326", inplace=True)

        dados_recorte = dados.rio.clip(bacia.geometry, bacia.crs, drop=True)

        for lat in dados_recorte.lat.values:
            for lon in dados_recorte.lon.values:
                try:
                    ponto = dados_recorte.sel(lat=lat, lon=lon, method="nearest")
                    df_ponto = ponto.to_dataframe().reset_index().dropna(subset=[variavel])

                    if df_ponto.empty:
                        continue

                    nome_ponto = f"{nome_modelo}_{round(lat, 4)}_{round(lon, 4)}.csv"
                    caminho_csv = os.path.join(pasta_modelo, nome_ponto)
                    df_ponto.to_csv(caminho_csv, index=False)

                except Exception as e:
                    print(f"Erro no ponto lat={lat}, lon={lon}: {e}")

        print(f">>> Finalizado: {nome_modelo}")

    except Exception as e:
        print(f"Erro ao processar {caminho_nc}: {e}")

In [ ]:
# ------------------------------------------------------------------------------
# Script: Padronizar Séries Temporais Extraídas de Arquivos NetCDF (.nc)
# ------------------------------------------------------------------------------
# Descrição:
# Este script processa arquivos CSV gerados a partir de extrações de dados 
# climáticos (ex: precipitação, temperatura) contidos em arquivos NetCDF.
# Ele reorganiza os dados no formato padrão: Dia;Mes;Ano;Valor.
#
# Funcionalidades:
# - Detecta automaticamente o separador do CSV.
# - Identifica a coluna de tempo e a coluna da variável (ex: 'pr').
# - Extrai Dia, Mês, Ano da coluna de tempo.
# - Regride 80 anos nos dados (ex: 2015 → 1935).
# - Salva os arquivos formatados em uma nova subpasta chamada "input", 
#   espelhando a estrutura de pastas original.
#
# Requisitos:
# - Pacotes: pandas, pathlib, os
#
# Autor: Matheus Marinho
# Projeto: Mestrado - Modelagem Hidrológica com o MGB e Projeções CLIMBra
# Data: Junho/2025
# ------------------------------------------------------------------------------


import os
import pandas as pd
from pathlib import Path

# === CONFIGURAÇÃO ===
BASE_DIR   = Path(r"E:\CLIMBRA_SSP585\Tmin")
OUTPUT_DIR = BASE_DIR / "input"

def verificar_csv(path: Path) -> bool:
    if path.stat().st_size == 0:
        print(f"Aviso: {path} está vazio.")
        return False
    return True

def process_csv(path: Path):
    if not verificar_csv(path):
        return

    # Tenta detectar separador automaticamente
    df = None
    for sep in [',',';','\t']:
        try:
            tmp = pd.read_csv(path, sep=sep)
            if tmp.shape[1] >= 2:
                df = tmp
                break
        except Exception:
            continue
    if df is None:
        print(f"Falha ao ler {path} com delimitadores padrão.")
        return

    # Detecta a coluna de tempo e a de valor
    col_time = next((col for col in df.columns if "time" in col.lower()), df.columns[0])
    col_valor = next((col for col in df.columns if col.lower() in ["pr", "valor", "value"]), df.columns[-1])

    # Converter datas
    df[col_time] = pd.to_datetime(df[col_time], errors='coerce')
    df_out = pd.DataFrame({
        'Dia':   df[col_time].dt.day,
        'Mes':   df[col_time].dt.month,
        'Ano':   df[col_time].dt.year,
        'Valor': df[col_valor]
    })

    # Regridar 80 anos
    df_out['Ano'] = df_out['Ano'] - 80

    # Caminho de saída
    rel    = path.relative_to(BASE_DIR)
    target = OUTPUT_DIR / rel
    target.parent.mkdir(parents=True, exist_ok=True)

    # Salvar CSV
    df_out.to_csv(target, sep=';', index=False)
    print(f"✔️ Salvo: {target}")

def processar_todos():
    OUTPUT_DIR.mkdir(exist_ok=True)
    for root, _, files in os.walk(BASE_DIR):
        root_path = Path(root)
        if OUTPUT_DIR == root_path or OUTPUT_DIR in root_path.parents:
            continue
        for fname in files:
            if fname.lower().endswith('.csv'):
                process_csv(root_path / fname)

if __name__ == "__main__":
    processar_todos()


In [ ]:
# ------------------------------------------------------------------------------
# Script: Ajustar Datas Incompletas em Séries Temporais (Anos Bissextos + Dia 31)
# ------------------------------------------------------------------------------
# Descrição:
# Este script percorre arquivos CSV contendo séries temporais no formato 
# Dia;Mes;Ano;Valor e insere automaticamente valores faltantes para:
# - 29 de fevereiro em anos bissextos, caso não esteja presente;
# - Dias 31 nos meses que possuem 31 dias, caso ausentes.
#
# A linha inserida recebe o valor 0.0 e a posição correta na sequência temporal.
#
# Funcionalidades:
# - Lê os arquivos CSV a partir de um diretório de entrada (INPUT_DIR).
# - Detecta e preenche datas faltantes com zero.
# - Recria a estrutura de subpastas no diretório de saída (OUTPUT_DIR).
# - Salva os arquivos corrigidos mantendo o nome original.
#
# Requisitos:
# - Python 3.7+
# - Pacotes: pandas, calendar, pathlib
#
# Autor: Matheus Marinho
# Projeto: Mestrado - Modelagem Hidrológica com o MGB e Projeções CLIMBra
# Data: Junho/2025
# ------------------------------------------------------------------------------

import pandas as pd
import calendar
from pathlib import Path

# ——————————————————————————————
# Ajuste estes paths
INPUT_DIR  = Path(r"E:\CLIMBRA_SSP585\Tmin\input")
OUTPUT_DIR = Path(r"E:\CLIMBRA_SSP585\Tmin_input")
#————————————————————————————————————————

# lista de meses com 31 dias
meses_31 = [1, 3, 5, 7, 8, 10, 12]

# percorre todos os CSVs dentro de INPUT_DIR (incluindo subpastas)
for arquivo in INPUT_DIR.rglob("*.csv"):
    # lê o CSV, pulando o header original
    df = pd.read_csv(
        arquivo,
        sep=';',
        skiprows=1,
        names=['dia','mes','ano','valor'],
        dtype={'dia':int, 'mes':int, 'ano':int, 'valor':float}
    )
    # monta coluna datetime
    df['data'] = pd.to_datetime({
        'year':  df['ano'],
        'month': df['mes'],
        'day':   df['dia']
    })

    insercoes = []
    for ano in sorted(df['ano'].unique()):
        # 29/02 em ano bissexto
        if calendar.isleap(ano) and not ((df['ano']==ano)&(df['mes']==2)&(df['dia']==29)).any():
            insercoes.append({
                'dia': 29, 'mes': 2, 'ano': ano,
                'valor': 0.0,
                'data': pd.Timestamp(year=ano, month=2, day=29)
            })
        # dia 31 nos meses de 31 dias
        for mes in meses_31:
            if not ((df['ano']==ano)&(df['mes']==mes)&(df['dia']==31)).any():
                insercoes.append({
                    'dia': 31, 'mes': mes, 'ano': ano,
                    'valor': 0.0,
                    'data': pd.Timestamp(year=ano, month=mes, day=31)
                })

    # concatena e ordena
    if insercoes:
        df = pd.concat([df, pd.DataFrame(insercoes)], ignore_index=True)
    df = df.sort_values('data').reset_index(drop=True)

    # determina onde salvar, recriando a estrutura de pastas
    rel_path = arquivo.relative_to(INPUT_DIR)            # ex: "ACCESS-…/file.csv"
    out_dir  = OUTPUT_DIR / rel_path.parent              # mantém subpasta
    out_dir.mkdir(parents=True, exist_ok=True)           # cria, se necessário

    # salva apenas as colunas originais
    df[['dia','mes','ano','valor']].to_csv(
        out_dir / arquivo.name,
        sep=';', index=False, float_format="%.1f"
    )

    print(f"Processado: {rel_path} → inseridas {len(insercoes)} linhas")

In [ ]:
# -*- coding: utf-8 -*-
"""
Geração do arquivo climático ASCII para o MGB a partir de:
Tmed, Umidade Relativa, Horas de Sol, Vento 10m e Pressão Atmosférica.

Entradas:
  E:\CLIMBRA_SSP245\Tmed_output
  E:\CLIMBRA_SSP245\hur_input
  E:\CLIMBRA_SSP245\rss_output
  E:\CLIMBRA_SSP245\sfc_output

Saída:
  E:\CLIMBRA_SSP245\clima_ascii\<MODEL>\<arquivo>.txt
"""

from pathlib import Path
import pandas as pd
import numpy as np
import re

# =======================
# CONFIGURAÇÕES
# =======================

BASE_TMED = Path(r"E:\CLIMBRA_SSP585\Tmed_output")
BASE_HUR  = Path(r"E:\CLIMBRA_SSP585\hur_input")
BASE_SUN  = Path(r"E:\CLIMBRA_SSP585\rss_output")
BASE_WIND = Path(r"E:\CLIMBRA_SSP585\sfc_output")

BASE_OUT  = Path(r"E:\CLIMBRA_SSP585\clima_ascii")
BASE_OUT.mkdir(parents=True, exist_ok=True)

# Espaçamento ASCII
COL_SPACE = [6, 5, 5, 11, 11, 11, 11, 11]

# Colunas finais
COLS = ["dia", "mes", "ano", "TP2M", "UR2M", "W10M", "SunHours", "PSLC"]

# Pressão fixa
PRESSAO_FIXA = 920.0

# Regex para extrair modelo, variável, cenário, lat, lon
# Ex.: ACCESS-CM2-tasmax-ssp245_-25.125_-49.125_Tmed.csv
NAME_REGEX = re.compile(
    r"^(?P<model>.+?)-(?P<var>[^-]+)-(?P<scen>ssp\d+)_"
    r"(?P<lat>-?\d+(?:\.\d+)?)_(?P<lon>-?\d+(?:\.\d+)?).*\.csv$"
)


# =======================
# FUNÇÕES AUXILIARES
# =======================

def parse_name_info(filename: str):
    m = NAME_REGEX.match(filename)
    if not m:
        raise ValueError(f"Nome não segue padrão esperado: {filename}")
    return m.group("model"), m.group("var"), m.group("scen"), m.group("lat"), m.group("lon")


def find_matching_file(base_dir: Path, model: str, scen: str, lat: str, lon: str,
                       prefer_keyword: str | None = None):
    """
    Procura recursivamente em base_dir um arquivo com:
      <model>-QUALQUER-<scen>_<lat>_<lon>.csv
    Se prefer_keyword for dado, prioriza arquivos cujo nome contenha essa palavra.
    """
    pattern = f"{model}-*-{scen}_{lat}_{lon}*.csv"
    candidates = list(base_dir.rglob(pattern))
    if not candidates:
        raise FileNotFoundError(f"Nenhum arquivo encontrado em {base_dir} com padrão {pattern}")

    if prefer_keyword:
        preferred = [c for c in candidates if prefer_keyword.lower() in c.name.lower()]
        if preferred:
            return preferred[0]
    # Se não houver preferido, devolve o primeiro
    return candidates[0]


def carregar_csv(path: Path, var_name: str, candidates: list[str]):
    """
    Lê CSV com separador ';' e renomeia a primeira coluna encontrada em 'candidates'
    para var_name.
    """
    df = pd.read_csv(path, sep=";")

    # Ajusta possíveis nomes de mes/ano (mes vs mês)
    colmap = {}
    for c in df.columns:
        if c.lower() == "mês":
            colmap[c] = "mes"
    if colmap:
        df = df.rename(columns=colmap)

    # Garante que tenhamos dia, mes, ano
    for col in ["dia", "mes", "ano"]:
        if col not in df.columns:
            raise KeyError(f"Coluna '{col}' não encontrada em {path}")

    col_val = None
    for cand in candidates:
        for c in df.columns:
            if c.lower() == cand.lower():
                col_val = c
                break
        if col_val:
            break

    if col_val is None:
        raise ValueError(f"Não encontrei nenhuma das colunas {candidates} em {path}")

    df = df[["dia", "mes", "ano", col_val]].copy()
    df = df.rename(columns={col_val: var_name})
    return df


def formatar_ascii(df: pd.DataFrame) -> str:
    """
    Formata o DataFrame final para string ASCII com espaçamento COL_SPACE e
    1 casa decimal nas colunas numéricas (exceto dia/mes/ano).
    """
    df_fmt = df.copy()
    # Tipos
    for col in ["dia", "mes", "ano"]:
        df_fmt[col] = df_fmt[col].astype(int)

    for col in ["TP2M", "UR2M", "W10M", "SunHours", "PSLC"]:
        df_fmt[col] = df_fmt[col].astype(float).map(lambda x: f"{x:0.1f}")

    # to_string com espaçamento
    txt = df_fmt.to_string(
        col_space=COL_SPACE,
        justify="right",
        index=False,
        header=False
    )
    return txt


# =======================
# PROCESSAMENTO DE UM MODELO
# =======================

def processar_modelo(model_folder: str):
    print(f"\n📌 Processando modelo: {model_folder}")

    tmed_dir = BASE_TMED / model_folder
    if not tmed_dir.exists():
        print(f"  ⚠ Pasta Tmed não encontrada para {model_folder}")
        return

    tmed_files = sorted(tmed_dir.glob("*.csv"))
    if not tmed_files:
        print(f"  ⚠ Nenhum arquivo Tmed encontrado em {tmed_dir}")
        return

    for f_tmed in tmed_files:
        fname = f_tmed.name
        try:
            model, var, scen, lat, lon = parse_name_info(fname)
        except ValueError as e:
            print(f"  ⚠ Pulando arquivo com nome incompatível: {fname} ({e})")
            continue

        # Localiza arquivos das outras variáveis
        try:
            f_hur  = find_matching_file(BASE_HUR,  model, scen, lat, lon, prefer_keyword="hur")
            f_sun  = find_matching_file(BASE_SUN,  model, scen, lat, lon, prefer_keyword="rss")
            f_wind = find_matching_file(BASE_WIND, model, scen, lat, lon, prefer_keyword="sfc")
        except FileNotFoundError as e:
            print(f"  ❌ {e}")
            continue

        print(f"  Arquivo Tmed : {f_tmed}")
        print(f"  Arquivo HUR  : {f_hur}")
        print(f"  Arquivo SUN  : {f_sun}")
        print(f"  Arquivo WIND : {f_wind}")

        # Carrega as quatro variáveis
        df_tmed = carregar_csv(f_tmed, "TP2M",    ["tmed", "Tmed", "tp2m", "valor"])
        df_hur  = carregar_csv(f_hur,  "UR2M",    ["hur", "hurs", "ur2m", "valor"])
        df_sun  = carregar_csv(f_sun,  "SunHours",["n_h_clip", "sunhours", "n_h", "valor"])
        df_wind = carregar_csv(f_wind, "W10M",    ["u10", "w10m", "vento", "sfcwind", "valor"])

        # Merge geral
        df = df_tmed.merge(df_hur,  on=["dia", "mes", "ano"]) \
                    .merge(df_sun,  on=["dia", "mes", "ano"]) \
                    .merge(df_wind, on=["dia", "mes", "ano"])

        # Pressão fixa
        df["PSLC"] = PRESSAO_FIXA

        # Ordena
        df = df.sort_values(by=["ano", "mes", "dia"]).reset_index(drop=True)

        # Reorganiza colunas
        df_final = df[COLS].copy()

        # ASCII
        ascii_str = formatar_ascii(df_final)

        # Saída
        out_dir = BASE_OUT / model_folder
        out_dir.mkdir(parents=True, exist_ok=True)

        out_file = out_dir / fname.replace(".csv", ".txt")
        with open(out_file, "w") as f_out:
            f_out.write(ascii_str)

        print(f"  ✔ Arquivo gerado: {out_file}")


# =======================
# EXECUÇÃO PRINCIPAL
# =======================

def main():
    modelos = [d.name for d in BASE_TMED.iterdir() if d.is_dir()]

    if not modelos:
        print(f"⚠ Nenhuma pasta de modelo encontrada em {BASE_TMED}")
        return

    for m in modelos:
        processar_modelo(m)

    print("\n🎉 Finalizado com sucesso!\n")


if __name__ == "__main__":
    main()

In [ ]:
# ------------------------------------------------------------------------------
# Script: Renomear Arquivos ASCII do MGB com Códigos de Grade (Grid.csv)
# ------------------------------------------------------------------------------
# Descrição:
# Este script percorre arquivos `.txt` gerados para o modelo MGB, localiza os
# nomes que contêm coordenadas geográficas (lat/lon) e renomeia os arquivos
# com o código correspondente (campo 'Codigo') de 8 dígitos conforme planilha
# de mapeamento fornecida (`Grid.csv`).
#
# Funcionalidades:
# - Lê a tabela de mapeamento com latitudes, longitudes e códigos do grid.
# - Identifica lat/lon no nome dos arquivos via expressão regular.
# - Aplica tolerância para casar coordenadas com diferentes casas decimais.
# - Renomeia cada arquivo para o código numérico correspondente (ex: 00012345.txt).
# - Mantém a estrutura de diretórios e ignora nomes que não seguem o padrão.
#
# Requisitos:
# - Python 3.7+
# - Pacotes: pandas, pathlib, re
#
# Autor: Matheus Marinho
# Projeto: Mestrado - Modelagem Hidrológica com o MGB e Projeções CLIMBra
# Data: Junho/2025
# ------------------------------------------------------------------------------


import re
from pathlib import Path

import pandas as pd

# ——— Ajuste estes caminhos ———
OUTPUT_DIR   = Path(r"E:\CLIMBRA_SSP585\clima_ascii")
MAPPING_FILE = Path(r"E:\IGUAÇU_OTTO\Grid.csv")
# ——————————————————————————

# 1) Carrega o mapeamento (sep=';' e colunas Lat, Long, Codigo)
df_map = pd.read_csv(MAPPING_FILE, sep=';')
df_map['lat']  = df_map['Lat'].astype(float)
df_map['lon']  = df_map['Long'].astype(float)
df_map['code'] = df_map['Codigo'].astype(str).str.zfill(8)

# 2) Regex para extrair modelo, lat e lon do nome (sem extensão)
pattern = re.compile(r"(?P<model>.+)_(?P<lat>[-\d\.]+)_(?P<lon>[-\d\.]+)(?:_.*)?$")


# 3) Tolerância para casar truncamentos (2 casas vs 3 casas decimais)
tol = 1e-2

# 4) Percorre todos os .txt em OUTPUT_DIR
for txt_path in OUTPUT_DIR.rglob("*.txt"):
    stem = txt_path.stem
    m = pattern.match(stem)
    if not m:
        print(f"[AVISO] nome fora do padrão, pulando: {txt_path.name}")
        continue

    # 5) Extrai lat/lon do nome
    lat = float(m.group('lat'))
    lon = float(m.group('lon'))

    # 6) Busca o código dentro da tolerância definida
    cond_lat = (df_map['lat'] - lat).abs() < tol
    cond_lon = (df_map['lon'] - lon).abs() < tol
    match    = df_map[cond_lat & cond_lon]
    if match.empty:
        print(f"[ERRO] sem código para lat={lat}, lon={lon} em {txt_path.name}")
        continue
    code = match['code'].iloc[0]

    # 7) Renomeia o arquivo
    new_name = f"{code}.txt"
    new_path = txt_path.with_name(new_name)
    if new_path != txt_path:
        txt_path.rename(new_path)
        print(f"{txt_path.name} → {new_name}")
    else:
        print(f"{txt_path.name} já está com nome correto")